In [19]:
# setup
from IPython.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Greedy Algorithms, Unit-Task Scheduling, Knapsack


Agenda:

- Framework for Greedy algorithms
- Unit-task Scheduling
- The Knapsack problem

Resources: This information comes from Cormen et al. [introduction to algorithms](https://monet.en.kku.ac.th/courses/EN812303/book/Introduction.to.Algorithms.4th.Edition.pdf).


## The Greedy Paradigm

We previously looked at dynamic programming, where we solved a problem optimally by considering every subproblem generated from every possible choice.

We will now look at *Greedy* algorithms that save us from having to consider every possible subproblem when our problem comes with a special structure.

The greedy framework is very simple: 

- Let $\mathcal{X}$ be possible choices for the solution. Initialize solution $S=\emptyset$. 
- Select $x\in\mathcal{X}$ according to a greedy criterion $C(x)$ and set $S := S \cup \{x\}, \mathcal{X} := \mathcal{X} - \{x\}$.
- Repeat until solution is complete.

Selection Sort was an example of a greedy strategy. What was the greedy criterion? Why was our algorithm correct?



For selection sort, we can see that swapping minimum element into the current position is a correct choice for the "optimal" solution. Moreover, selecting a minimum and sorting the rest of the list recursively is also correct.

In [20]:
def selection_sort(L):
    if (len(L) == 1):
        return(L)
    else:
        m = L.index(min(L))
        print('selecting minimum %s' % L[m])       
        L[0], L[m] = L[m], L[0]
        print('recursively sorting L=%s\n' % L[1:])
        return [L[0]] + selection_sort(L[1:])
    
selection_sort([2, 1, 999, 4, 3])

selecting minimum 1
recursively sorting L=[2, 999, 4, 3]

selecting minimum 2
recursively sorting L=[999, 4, 3]

selecting minimum 3
recursively sorting L=[4, 999]

selecting minimum 4
recursively sorting L=[999]



[1, 2, 3, 4, 999]

<h3>Dijkstra's Revisited</h3>

Recall Dijkstra's algorithm: To find the shortest paths between a source vertex and all other reachable vertices in an edge-weighted graph, we greedily choose the closest vertex on the frontier and expand the frontier. It can be considered as a greedy algorithm because there is a sequence of choices (which vertex to visit next) and we choose by minimizing the distance to the source. We have already proven the correctness of Dijkstra's algorithm. Later, we will generalize the argument using Greedoids/Matroids.

## When is it correct to be Greedy?

When would the greedy strategy yield the correct solution?

1. **Optimal substructure**: An optimal solution can be constructed from optimal solutions of successively smaller subproblems.
2. **Greedy choice**: A greedy choice must be in some optimal solution. In other words, at every step, by making the greedy choice, we are building up an optimal solution.



Put together, these two properties yield that the iterative strategy above constructs an optimal solution.



Due to their simplicity greedy algorithms are easy to implement. However proving the optimal substructure and greedy choice properties requires a problem-specific approach and can be tricky. 

We will see that many greedy algorithms can be identified by the existence of a <i>matroid</i> structure. A common [misconception](https://nor-blog.codeberg.page/posts/2023-01-04-greedoids/) is that every greedy algorithm comes from a matroid (or the related greedoid) structure. The next example demonstrates that this is not necessarily the case.

But we will see that most greedy algorithms do have such a structure. In later classes, we will explore this further.

### Unit Task Scheduling

Consider the **unit task scheduling problem** where we are given a set of $n$ tasks $A = \{a_0, \ldots, a_{n-1}\}$. Each task $a_i$ has start and finish times $(s_i, f_i)$. The goal is to select a subset $S$ of tasks with no overlaps that is as large as possible.


<br>

**Example**: Let the tasks be $a_0=(0, 2), a_1=(2, 4), a_2=(4, 6), a_3=(2, 8)$. 

What is the largest set of non-overlapping tasks?



<img src="figures/simple_unit_task.jpg" width="30%">


We should choose $S = \{a_0, a_1, a_2\}$ since choosing $a_3$ "blocks" a large part of the solution.


## A greedy solution?

Is there a greedy algorithm for the unit task scheduling problem?

Does this problem have optimal substructure? What should the greedy choice be?


<br>

**Optimal Substructure:** Since the tasks are nonoverlapping, if we identify one task in the optimal solution we can eliminate the tasks overlapping it and recursively solve for the remaining tasks.

<br>


Is there a greedy criterion with the greedy choice property?

### Greedy Criteria

What if we chose the shortest task first?


Counterexample:
<img src="./figures/shortest_first_counterexample.jpg" width=30%>


What if we chose the task that started earliest? 


Counterexample:
<img src="./figures/earliest_first_counterexample.jpg" width=30%>


What if we successively chose the task with fewest overlaps?


Counterexample:
<img src="./figures/minimum_overlap_counterexample.jpg" width=30%>


What if we chose the task that finished earliest?

Counterexample:
<img src="figures/unit-task-all.png" width="60%">



This works on all of our counterexamples.



From our original example:

$\pmb{a_0=(0, 2), a_1=(2, 4), a_2=(4, 6)}, a_3=(2, 8)$


<br>


What would the greedy choice property say here?



**Greedy Choice Property**: For any set of tasks, the task with earliest finish time is in some optimal solution.
 

## Proof

**Greedy Choice Property**: For any set of tasks, the task with earliest finish time is in some optimal solution.

**Proof**: Given a set of tasks $A$, let $G$ be the greedy solution and let $O$ be an optimal solution. 



<img src="./figures/greedy_choice_unit_task_proof.jpg" width="60%">


Now, suppose that $G\neq O$. Sort the tasks by finish time, and let tasks $a\in G$ and $a'\in O$ be the first pair of corresponding tasks with $a\neq a'$.

If such a pair does not exist, there are a few options:
1. $G=O$ (We are done)
2. $G\subset O$ (Then add $a'$ to $G$)
3. $O\subset G$ (This can't happen because $O$ is optimal)

By the definition of our greedy choice, this implies that $a$ finishes before $a'$.




So we have that $G = \langle G_1, a, G_2\rangle$ and $O = \langle O_1, a', O_2\rangle$.


Consider the solution $X = \langle O_1, a, O_2\rangle$. 

Since $a$ and $a^\prime$ are the first pair of tasks that differ, $G_1=O_1$, and because $a$ is compatible with $G_1$, $a$ must also be compatible with $O_1$.



Since $O$ is a valid solution and $a$ has earlier finish time than $a'$, all the tasks in $O_2$ are compatible with $\langle G_1, a\rangle$. This demonstrates that $X$ is a valid solution.


The value of $X$ is $|X| = |O_1|+ |O_2| +1 = |O|$ so $X$ is also an optimal solution. Moreover, $X$ and $G$ agree on one more task than $O$ and $G$. This means that we can repeat this procedure after setting $O=X$. If we repeat $|O|$ times, we will find that $G=X$, so $G$ is optimal. 

$\blacksquare$


In [28]:
objects = [(0,2),(2,4),(4,6),(2,8)]
tasks_sched=[]#Will hold our solution. 
def schedule_tasks(objects):
    sorted_objects= sorted(objects, key=lambda obj: obj[1])#Sort by second coordinate.
    last_end_time = -1
    for obj in sorted_objects:
        if obj[0] >= last_end_time:
            last_end_time = obj[1]
            tasks_sched.append(obj)
    #ASCII printout:
    for obj in objects:
        if obj in tasks_sched:
            line = " "*obj[0]+"="*(obj[1]-obj[0])
        else:
            line = " "*obj[0]+"-"*(obj[1]-obj[0])
        print(line)
schedule_tasks(objects)

==
  ==
    ==
  ------
 --


## Runtime Analysis

So we've proven that the earliest finish time first strategy produces the optimal solution. What about work/span?
 




Given a set of tasks, we can sort by earliest finish time in $O(n\log n)$ work and $O(\log n)$ span. 



Then, we sequentially step through the tasks, adding a task to the solution if it doesn't overlap with a task that is already chosen. This takes $O(n)$ work/span. Generally speaking, greedy algorithms cannot be parallelized easily because they require sequential choices.

$$W(n) \in O(n \log n)$$

$$S(n) \in O(n)$$

Greedy algorithms are typically very fast. Generally speaking, they should take linear or nearly linear work and span as in this case. For comparison, observe that the scheduling problem is a special case of calculating the independence number of a graph, for which no polynomial-time algorithm is known. When greedy algorithms apply, they are typically very fast.

### The Knapsack Problem Revisited

Let's look another optimization problem. Suppose there are $n$ objects, each with a *value* $v_i>0$ and *weight* $w_i>0$. You have a "knapsack" of capacity $W\geq 0$ and want to fill it with a set of objects $X \subseteq [n]$ so that $w(X) \leq W$ and $v(X)$ is maximized. 



Example: Suppose you have 3 objects with values/weights $(10, 5), (6, 3), (6, 2)$ and $W=5$. 


<style>.jp-RenderedHTMLCommon table {
  border-collapse: collapse;
  border-spacing: 0;
  border: none;
  color: var(--jp-ui-font-color1);
  font-size: 20px;
  table-layout: fixed;
  margin-left: auto;
  margin-right: auto;
}</style>

|item  |value |weight|
|------|------|-----|
|$a_0$ | 10   | 5   |
|$a_1$ | 6    | 3   |
|$a_2$ | 6    | 2   |



How can we fill our knapsack to maximize its value?



## Greedy Choices

> **Example Problem**
>
> |item|value|weight|
> |------|------|-----|
> |$a_0$ |10    |5    |
> |$a_1$ |6     |3    |
> |$a_2$ |6     | 2   |
>
> Knapsack Capacity: $W=5$
>
> Optimal Choices: $X = [a_1, a_2]$, $v(X) = 12$ 



<br>

What if we greedily took the **most valuable** object first?

We would select item $a_0$ for a total value of $10$.


What if we greedily took the object with **highest per unit value** (i.e., maximum value/weight ratio)?



In the example above, we'd achieve a value of $12$, which is optimal. Does this always work?

Unfortunately no. Can we come up with a counter example for this choice?





Counterexample:

|item  |value|weight|
|------|------|-----|
|$a_0$ | 10   | 5   |
|$a_1$ | 9    | 3   |

Knapsack Capacity: $W=5$

Optimal Choices: $X = [a_1]$, $v(X) = 10$ 

$a_1$ has the highest value per weight, but it does not belong in the optimal solution.

<br>

Is there another greedy criterion?

Probably not, since the Knapsack problem is NP-complete, and a greedy solution would give a linear-time algorithm.




The problem is that a greedy algorithm, regardless of the criterion, selects objects one at a time, without factoring in the capacity remaining into future decisions.


## Fractional Knapsack

What if we wanted to solve the **fractional** version of the problem, where we could take fractional amounts of each item?



Does our weight/value greedy criterion satisfy the greedy choice property?



**Greedy Solution for Fractional Knapsack**: Sort the items by their value-to-weight ratio, $v_i/w_i$. Take as much the first item as will fit into the sack. Then continue to the next item until all items are exhausted or the sack is full. This greedy algorithm will give an optimal solution to the fractional knapsack problem.


**Proof**: Suppose we were given $n$ objects and their weights/values. Sort the items by their value-to-weight ratio so that $a_0$ has the largest value-to-weight ratio.



Let $O$ be the optimal solution, and let $G$ be the greedy solution. We denote the fraction of item $a_i$ taken in $G$ by $\alpha_i^G\in [0,1]$ and the fraction of $a_i$ taken in $O$ by $\alpha_i^O\in[0,1]$.


Let item $a_j$ have the maximum value/weight ratio such that $\alpha_j^O\neq \alpha_j^G$. Note that the greedy solution will either take all of $a_j$ so that $\alpha_j^G=1$ or will exhaust the capacity of the sack using $a_j$. In the second case, no other objects after $a_j$ will be taken.


Therefore, if $O\neq G$, we may assume of $O$ that both: 
1. $\alpha_j^O<\alpha_j^G\leq 1$ because: 
    - Due to the definition of the greedy algorithm, $\alpha_j^O\leq \alpha_j^G$. 
    - Due to the definition of $a_j$, $\alpha_j^O \neq \alpha_j^G$.
2. For some $i>j$, $\alpha_i^O>0$, because:
    - Since $\alpha_j^O<\alpha_j^G$, there is leftover capacity in $O$ after taking items $a_0$ to $a_j$. An optimal solution should take some amount of another item, else it is not optimal.


Since $v_i/w_i \leq v_j/w_j$,we can substitute some of item $a_i$ for item $a_j$ to get a solution that is at least as good.

Specifically, set $\bar{\alpha_i^{O}} =\max(\alpha_i^O, (1-\alpha_j^O)\frac{w_j}{w_i})$. This will be the amount of $a_i$ that we give up. Set $\bar{\alpha_j^{O}}=\bar{\alpha_i^{O}}\frac{w_i}{w_j}$. This will be the amount of $a_j$ that we gain.

This definition ensures that $\alpha_i^O-\bar{\alpha_i^{O}}\geq 0$ and $\alpha_j^O+\bar{\alpha_j^{O}}\leq 1$ so the solution that comes from performing this substitution is feasible.

We can also check that the weight of the new solution does not change, yet the value increases.

This shows that we can get a solution that is at least as good by increasing the amount of $a_j$ taken in the optimal solution. By repeating this argument many times (at most the number of items, $n$), we will arive at an optimal solution with the same value as the greedy solution. $\blacksquare$


**Implementation:**

How would we implement our solution? We sort by value/weight ratios and take successive items until we end with a possibly fractional item. The work is $O(n\log n + n) = O(n\log n)$ and the span is $O(\log^2 n + n) = O(n)$.